In [10]:
import nltk
from nltk.corpus import stopwords
from nltk import ngrams
import spacy
from nltk.tokenize import sent_tokenize, word_tokenize
import csv
import pandas as pd
import json
import time
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import string 
import unicodedata as ud
from greek_stemmer import GreekStemmer
import pymongo
import numpy as np
import re
import networkx as nx
import indexer as ind
mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")
#mongo_client.drop_database("GreekParliamentProceedings")
client = mongo_client["GreekParliamentProceedings"]
index = client["InvertedIndex"]
database = client["Database"]

C:\Users\harry\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Tests

In [11]:
import spacy
nlp = spacy.load('el_core_news_sm')

In [12]:

text = "εξαιρετικές"
doc = nlp(text)
for token in doc:
    print(token.pos_)  


ADJ


### Methods

In [3]:
from greek_stemmer import GreekStemmer

def get_candidate_words(text, stopwords, domain_specific_stopwords, language):
    stemmer = GreekStemmer()
    #STEP 1: SPLIT TEXT INTO SENTENCES AND PRE-PROCESS 
    nlp = language
    nlp.Defaults.stop_words |= stopwords
    sentences = nlp(text)
    processed_sentences = []
    for sentence in sentences.sents:
        words_to_consider = []
        #print(sentence)
        for token in sentence:
            if(token.pos_ in ('NOUN','ADJ') and token.is_stop==False and token.is_punct==False):
                d = {ord('\N{COMBINING ACUTE ACCENT}'):None}
                if(stemmer.stem(ud.normalize('NFD',token.text.upper()).translate(d)).lower() not in domain_specific_stopwords):
                    words_to_consider.append(token.text)
        if(len(words_to_consider)>0):
            processed_sentences.append(words_to_consider)
    return processed_sentences

In [4]:
def extract_pairs(word_list, window):
    pairs = []
    #print(word_list)
    g = nx.Graph()
    for a_list in word_list:
        if(len(a_list)>window):
            candidate_lists = [a_list[k:window+k] for k in range(0, len(a_list)) if window+k<=len(a_list)]
        else:
            candidate_lists = [a_list]
        for candidate_list in candidate_lists:
            #print(candidate_list)
            for i in range(0, len(candidate_list)-1):
                for j in range(i+1, len(candidate_list)):
                    pairs.append((candidate_list[i], candidate_list[j]))
                    #avoid self-loops
                    if(candidate_list[i] == candidate_list[j]):
                        pass
                    #if edge does not exist create it, else increase edge weight
                    elif((candidate_list[i], candidate_list[j]) not in g.edges):
                        g.add_edge(candidate_list[i], candidate_list[j])
                        g[candidate_list[i]][candidate_list[j]]['weight'] = 1
                    else:
                        g[candidate_list[i]][candidate_list[j]]['weight'] += 1
    return pairs, g

In [5]:
keywords_len = {}
import math
def document_keywords(speech, stopwords, domain_specific_stopwords, window, language):
    sentences = get_candidate_words(speech, stopwords, domain_specific_stopwords, language)
    pairs, graph = extract_pairs(sentences, window=window)
    scores = nx.pagerank(graph)
    scores = dict(sorted(scores.items(), key=lambda item: -item[1]))
    all_words = list(scores.keys())
    if(len(graph.nodes)>0):
        keywords = all_words[:int(math.log2(len(graph.nodes))+8)]
    else:
        keywords = []
    return scores, keywords

### Keywords for a single document

In [71]:
mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")
#mongo_client.drop_database("GreekParliamentProceedings")
client = mongo_client["GreekParliamentProceedings"]
index = client["InvertedIndex"]
database = client["Database"]

In [72]:
document = list(database.find({"_id":"9433"}, { "_id": 0, "speech": 1 }))
#speech = document[0]['speech']

In [13]:
#words_in_row = ind.preprocess_doc(document[0]['speech'], stopwords)
with open('stopwords.txt', encoding='utf-8') as file:
  stopwords = [line.rstrip() for line in file]
  stopwords = set(stopwords)
with open('domain-specific-stopwords.txt', encoding='utf-8') as file:
  domain_specific_stopwords = [line.rstrip() for line in file]
#language = spacy.load('el_core_news_sm')
#scores, keywords = document_keywords(speech, stopwords, domain_specific_stopwords, window = 3, language=language)
#print(keywords)
#scores, keywords = document_keywords(speech, stopwords, domain_specific_stopwords, window = 4)
#print(keywords)
#scores, keywords = document_keywords(speech, stopwords, domain_specific_stopwords, window = 2)
#print(keywords)


In [76]:
document = list(database.find({"_id":"1"}, { "_id": 0, "speech": 1 }))
speech = document[0]['speech']
#words_in_row = ind.preprocess_doc(document[0]['speech'], stopwords)
with open('stopwords.txt', encoding='utf-8') as file:
  stopwords = [line.rstrip() for line in file]
  stopwords = set(stopwords)
with open('domain-specific-stopwords.txt', encoding='utf-8') as file:
  domain_specific_stopwords = [line.rstrip() for line in file]
language = spacy.load('el_core_news_sm')
scores, keywords = document_keywords(speech, stopwords, domain_specific_stopwords, window = 3, language = language )
#scores, keywords = document_keywords(speech, stopwords, domain_specific_stopwords, window = 4)
print(speech)
#scores, keywords = document_keywords(speech, stopwords, domain_specific_stopwords, window = 2)
print(keywords)


 Παρακαλείται ο κύριος Γραμματέας να συνοδεύσει την Ιερά Σύνοδο εκτός της Αιθούσης της Βουλής.  . Παρακαλείται ο συνάδελφος Βουλευτής κ. Σαδίκ Αμέτ, που ανήκει στο Μωαμεθανικό Θρήσκευμα να προσέλθει και να δώσει τον οριζόμενο από το Σύνταγμα όρκο επί του Κορανίου.  : ~"Ορκίζομαι στο όνομα του Παντοδύναμου Θεού και του μόνου αυτού Προφήτη ο οποίος είναι ο Μωάμεθ να είμαι πιστός στην πατρίδα και το δημοκρατικό πολίτευμα, να υπακούω στο Σύνταγμα και τους νόμους και να εκπληρώνω ευσυνείδητα τα καθήκοντά μου".
['Σύνταγμα', 'πατρίδα', 'Παντοδύναμου', 'δημοκρατικό', 'ευσυνείδητα', 'οριζόμενο', 'όρκο', 'Ιερά', 'Σύνοδο', 'Αιθούσης', 'όνομα', 'πολίτευμα']


### Keywords per parliament member

In [77]:
#get distinct parliament member names in a list
member_names = database.distinct("member_name")
#years = database.distinct("year")

In [78]:
stemmer = GreekStemmer()
d = {ord('\N{COMBINING ACUTE ACCENT}'):None}
stemmed = stemmer.stem(ud.normalize('NFD',"πολυμεγαληλε".upper()).translate(d)).lower()

In [87]:
def extract_phrases(speech, keywords):
    tokenized = speech.split()
    bigrams = ngrams(tokenized, 2)
    for gram in bigrams:
        if(gram[0] in keywords and gram[1] in keywords and gram[0]!=gram[1]):
            keywords.append(gram[0]+" "+gram[1])
            keywords.remove(gram[0])
            keywords.remove(gram[1])
    return keywords
def stem_top_keywords(keywords:dict):
    #dict(keyword_frequency[:20])
    stemmer = GreekStemmer()
    d = {ord('\N{COMBINING ACUTE ACCENT}'):None}
    final = {}
    for token in keywords.keys():
        #split to tokens
        words = word_tokenize(token)
        stemmed = ""
        for word in words:
            #remove word if it is duplicate or already exists in keyphrase
            stemmed_word = stemmer.stem(ud.normalize('NFD',word.upper()).translate(d)).lower()
            stemmed = stemmed + stemmed_word
            if len(words)>1:
                stemmed = stemmed + " "
                if(stemmed_word in final.keys()):
                    final.pop(stemmed_word)            

        if(stemmed in final.keys()):
            final[stemmed] = final[stemmed]+keywords[token]
        else:
            final[stemmed] = keywords[token]
    return final

def get_member_keywords(member_name:string)->list:
    #get ids of member's total speeches
    member_speeches = list(database.find({"member_name": member_name}, { "_id": 0, "speech": 1 }))
    total_keywords = []
    language = spacy.load('el_core_news_sm')
    with open('stopwords.txt', encoding='utf-8') as file:
        stopwords = [line.rstrip() for line in file]
        stopwords = set(stopwords)
    with open('domain-specific-stopwords.txt', encoding='utf-8') as file:
        domain_specific_stopwords = [line.rstrip() for line in file]
    print(f"Member {member_name} has {len(member_speeches)} speeches.")
    counter = 0
    for member_speech in member_speeches:
        scores, member_keywords = document_keywords(member_speech['speech'], stopwords, domain_specific_stopwords, window = 3, language = language)
        member_keywords = extract_phrases(member_speech['speech'], member_keywords)
        total_keywords.extend(member_keywords)
        #print('Doc: ', counter)
        counter+=1
    keyword_frequency = {}
    for keyword in set(total_keywords):
        keyword_frequency[keyword] = total_keywords.count(keyword)
    keyword_frequency = sorted(keyword_frequency.items(), key=lambda d: d[1], reverse=True)
    number_of_keywords = int(math.log2(len(member_speeches)))
    top_keywords = stem_top_keywords(dict(keyword_frequency[:number_of_keywords]))
    return total_keywords, keyword_frequency, top_keywords

In [69]:
total_keywords, keyword_frequency, top_keywords = get_member_keywords(member_names[1])


Member αβδελας κωνσταντινου αποστολος has 2736 speeches.
Doc:  0
Doc:  1
Doc:  2
Doc:  3
Doc:  4
Doc:  5
Doc:  6
Doc:  7
Doc:  8
Doc:  9
Doc:  10
Doc:  11
Doc:  12
Doc:  13
Doc:  14
Doc:  15
Doc:  16
Doc:  17
Doc:  18
Doc:  19
Doc:  20
Doc:  21
Doc:  22
Doc:  23
Doc:  24
Doc:  25
Doc:  26
Doc:  27
Doc:  28
Doc:  29
Doc:  30
Doc:  31
Doc:  32
Doc:  33
Doc:  34
Doc:  35
Doc:  36
Doc:  37
Doc:  38
Doc:  39
Doc:  40
Doc:  41
Doc:  42
Doc:  43
Doc:  44
Doc:  45
Doc:  46
Doc:  47
Doc:  48
Doc:  49
Doc:  50
Doc:  51
Doc:  52
Doc:  53
Doc:  54
Doc:  55
Doc:  56
Doc:  57
Doc:  58
Doc:  59
Doc:  60
Doc:  61
Doc:  62
Doc:  63
Doc:  64
Doc:  65
Doc:  66
Doc:  67
Doc:  68
Doc:  69
Doc:  70
Doc:  71
Doc:  72
Doc:  73
Doc:  74
Doc:  75
Doc:  76
Doc:  77
Doc:  78
Doc:  79
Doc:  80
Doc:  81
Doc:  82
Doc:  83
Doc:  84
Doc:  85
Doc:  86
Doc:  87
Doc:  88
Doc:  89
Doc:  90
Doc:  91
Doc:  92
Doc:  93
Doc:  94
Doc:  95
Doc:  96
Doc:  97
Doc:  98
Doc:  99
Doc:  100
Doc:  101
Doc:  102
Doc:  103
Doc:  104
Doc

In [90]:
keywords_by_member = {}
for i in range(1, len(member_names)):
    total_keywords, keyword_frequency, top_keywords = get_member_keywords(member_names[i])
    keywords_by_member[member_names[i]] = top_keywords

Member αβδελας κωνσταντινου αποστολος has 2736 speeches.
Member αβραμακης αντωνιου ελευθεριος has 69 speeches.
Member αβραμιδης νικολαου γαβριηλ has 216 speeches.
Member αβραμοπουλος λαμπρου δημητριος has 701 speeches.
Member αγαθοπουλου δημητριου ειρηνη-ελενη has 105 speeches.
Member αγατσα αριστειδη αριαδνη has 22 speeches.
Member αγγελης στεφανου ανεστης has 174 speeches.
Member αγγελοπουλος κωνσταντινου νικολαος has 616 speeches.
Member αγγελουσης αθανασιου ευσταθιος has 4 speeches.
Member αγγουρακης γεωργιου χαραλαμπος has 552 speeches.
Member αγοραστος αναστασιου κωνσταντινος has 330 speeches.
Member αδαμοπουλος αθανασιου γεωργιος has 1059 speeches.
Member αδαμοπουλου αθανασιου αγγελικη has 69 speeches.
Member αδαμου στεργιου κωνσταντινα has 1 speeches.
Member αδρακτας φωτιου παναγιωτης has 599 speeches.
Member αηδονης ευαγγελου χρηστος has 483 speeches.
Member αθανασιαδης θεοδωρου αλεξανδρος has 67 speeches.
Member αθανασιος ηλιοπουλος has 58 speeches.
Member αθανασιος μικρουτσι